## Nucleus segmentation in 3D

+ Find all nd2 files and create a metadata file called 'sample.tsv'
+ Using stardist 2D segmentation and save the nucleus masks into folder 'nuclei_masks_2D/'
+ Link the 2D pieces into 3D object and save the 3D nucleus masks into folder 'nuclei_masks_3D_connect_centroid_3D/'

In [2]:
import sys
import numpy as np
from skimage import io
import numpy as np
# import nd2
# from skimage.filters import median
# from skimage.transform import rescale, resize
from stardist.models import StarDist2D
from csbdeep.utils import normalize # https://github.com/CSBDeep/CSBDeep/blob/master/csbdeep/utils/utils.py
import tifffile
# import matplotlib.pyplot as plt
# import seaborn as sns
import re
import pandas as pd
import os
from nd2reader import ND2Reader
from joblib import Parallel, delayed
from tqdm import tqdm
from scipy.ndimage import label
from scipy.spatial.distance import cdist
from tifffile import imwrite
import glob
from pathlib import Path
# from statannotations.Annotator import Annotator


2023-10-26 11:57:05.680097: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
## Define and change according to the experiments
pixel_size = 0.22 # um/px
z_step_size = 2 # um/px

image_folder = "230908" # "" for current folder

# Change to the image directory
os.chdir(image_folder)

In [3]:

# Find .nd2 files recursively
nd2_files = glob.glob('**/*.nd2', recursive=True)

# Create and open the output TSV file
with open('sample.tsv', 'w') as tsv_file:
    # Write the header row
    tsv_file.write("File\tGroup\tCondition\n")

    # Iterate through the found .nd2 files
    for nd2_file in nd2_files:
        # Extract the group and condition information from the directory structure
        parts = nd2_file.split(os.path.sep)
        group = parts[0]
        condition = ''

        # Write the data to the TSV file
        tsv_file.write(f"{nd2_file}\t{group}\t{condition}\n")



In [7]:
## read file description metadata

m_metadata_file = os.path.join("sample.tsv")
m_metadata_df = pd.read_csv(m_metadata_file,index_col=False,sep="\t")

m_metadata_df["image_short"] = [m_metadata_df["Group"][ith_file]+"_"+os.path.basename(os.path.dirname(m_metadata_df["File"][ith_file]))[-2:]+"_"+os.path.basename(m_metadata_df["File"][ith_file])[:-4] for ith_file in range(len(m_metadata_df["File"]))]
m_metadata_df

,File,Group,Condition,image_short
0,pd 2xl/40x_pERK488_atub555_NucBlue004/1.nd2,pd 2xl,NaN,pd 2xl_04_1
1,pd 2xl/40x_pERK488_atub555_NucBlue004/2.nd2,pd 2xl,NaN,pd 2xl_04_2
2,pd 2xl/40x_pERK488_atub555_NucBlue004/3.nd2,pd 2xl,NaN,pd 2xl_04_3
3,pd 2xl/40x_pERK488_atub555_NucBlue004/4.nd2,pd 2xl,NaN,pd 2xl_04_4
4,pd 2xl/40x_pERK488_atub555_NucBlue004/5.nd2,pd 2xl,NaN,pd 2xl_04_5
5,2XL/40x_pERK488_atub555_NucBlue/1.nd2,2XL,NaN,2XL_ue_1
6,2XL/40x_pERK488_atub555_NucBlue/2.nd2,2XL,NaN,2XL_ue_2
7,2XL/40x_pERK488_atub555_NucBlue/3.nd2,2XL,NaN,2XL_ue_3
8,2XL/40x_pERK488_atub555_NucBlue/4.nd2,2XL,NaN,2XL_ue_4
9,2XL/40x_pERK488_atub555_NucBlue/5.nd2,2XL,NaN,2XL_ue_5


In [4]:
path_segmentation = "nuclei_masks_2D"
Path(path_segmentation).mkdir(parents=True, exist_ok=True)
centroid_distance_threshold = 4.0/pixel_size # 1.5/pixel_size # 4.0/pixel_size # 4um centroids distance
volume_threshold = 100 # unit: um3
area_threshold = 20 # unit: um2
intensity_threshold = 200 # 5% of the intensity dynamic range
path_segmentation_3D_connect = "nuclei_masks_3D_connect_centroid_3D"
Path(path_segmentation_3D_connect).mkdir(parents=True, exist_ok=True)

In [22]:
# creates a pretrained model
model = StarDist2D.from_pretrained('2D_versatile_fluo')

Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.


In [23]:
def process_file(file_name,output_pre):
    #file_name = "../" + m_metadata_df["File"][ith_file]
    with ND2Reader(file_name) as img:
    #img = ND2Reader(file_name)
        img.bundle_axes = 'zyx'
        img.iter_axes = 'c'
    #     output_pre = path_segmentation + m_metadata_df["image_short"][ith_file]
        img_0 = img[0]
        img_0_min = np.min(img_0)
        img_0_max = np.max(img_0)
        img_0_norm = (img_0-img_0_min)/(img_0_max-img_0_min)
        for z_plane in range(len(img_0_norm)):
            output_pre_z = output_pre + "_z" + '{:02}'.format(z_plane)
            if os.path.exists(output_pre_z+'.tif'):
                labels = io.imread(output_pre_z+'.tif')
            else:
                if img_0_norm[z_plane].shape[0]> 2000:
                    labels, details = model.predict_instances_big(img_0_norm[z_plane],axes='YX',block_size=1024,min_overlap=200,context=200,show_progress=False,labels_out_dtype=np.uint16)
                else:
                    labels, details = model.predict_instances(img_0_norm[z_plane])
            # filter based on size and intensity
            object_list = np.unique(labels)[1:]
            labels_filter = np.zeros_like(labels)
            for ith_object in object_list:
                tmp_intensity = np.mean(img_0_norm[z_plane][labels==ith_object])
                tmp_size = np.sum(labels==ith_object)
                if tmp_intensity > intensity_threshold and tmp_size > area_threshold:
                    labels_filter[labels==ith_object] = ith_object
            # save the result
            io.imsave(output_pre_z+'.tif',labels_filter.astype('uint16'))
#             imwrite(output_pre_z+'.tif', labels_filter.astype('uint16'), shape=labels_filter.shape,\
#                     metadata={'axes': 'YX'}, imagej=True, photometric='minisblack', dtype='uint16', compression ='zlib')
            
num_threads = 4  # Number of threads to use

# Use joblib to run the process_file function in parallel with 8 threads
Parallel(n_jobs=num_threads)(
    delayed(process_file)(file_name = os.path.join(m_metadata_df["File"][ith_file]), output_pre=os.path.join(path_segmentation,m_metadata_df["image_short"][ith_file])) for ith_file in tqdm(range(len(m_metadata_df["File"].values)))
)

# # single thread
# for ith_file in tqdm(range(len(m_metadata_df["File"].values))):
#     process_file(file_name = os.path.join("..",m_metadata_df["File"][ith_file]), output_pre=os.path.join(path_segmentation,m_metadata_df["image_short"][ith_file]))






  0%|                                                    | 0/21 [00:00<?, ?it/s]

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers
......concatenate
.........vars
......concatenate_1
.........vars
......concatenate_2
.........vars
......conv2d
.........vars
............0
............1
......conv2d_1
.........vars
............0
............1
......conv2d_10
.........vars
............0
............1
......conv2d_11
.........vars
............0
............1
......conv2d_12
.........vars
............0
............1
......conv2d_13
.........vars
............0
............1
......conv2d_14
.........vars
............0
............1
......conv2d_15
.........vars
............0
............1
......conv2d_16
.........vars
............0
............1
......conv2d_17
.........vars
............0
............1
......conv2d_18
.........vars
............0
............1
......conv2d_2
.........vars
............0
............1
......conv2d_3
.........vars
............0
............1
......conv2d_4
.........vars
............0
............1
......conv2d_5
....

Keras model archive saving:
File Name                                             Modified             Size
variables.h5                                   2023-10-26 12:31:01      5775480
metadata.json                                  2023-10-26 12:31:01           64
config.json                                    2023-10-26 12:31:01        15111


2023-10-26 12:31:02.123839: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-26 12:31:02.256928: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-26 12:31:02.388348: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2
To enable them in other 

Keras model archive loading:
File Name                                             Modified             Size
variables.h5                                   2023-10-26 12:31:00      5775480
metadata.json                                  2023-10-26 12:31:00           64
config.json                                    2023-10-26 12:31:00        15111
Keras model archive loading:
File Name                                             Modified             Size
variables.h5                                   2023-10-26 12:31:00      5775480
metadata.json                                  2023-10-26 12:31:00           64
config.json                                    2023-10-26 12:31:00        15111


...

effective: block_size=(1024, 1024), min_overlap=(208, 208), context=(208, 208)
1/1 [==============================] - 0s 322ms/step


4047142328.py (30): nuclei_masks_2D/dmso 2xl_05_5_z12.tif is a low contrast image


effective: block_size=(1024, 1024), min_overlap=(208, 208), context=(208, 208)
1/1 [==============================] - 0s 315ms/step


4047142328.py (30): nuclei_masks_2D/dmso 2xl_05_5_z13.tif is a low contrast image


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [6]:
# Function to calculate centroid coordinates for a labeled region
def calculate_centroid(region):
    rows, cols = np.nonzero(region)
    centroid_x = np.mean(cols)
    centroid_y = np.mean(rows)
    return centroid_x, centroid_y

# Function to find closest centroid in neighboring images
def find_closest_centroid(current_centroid, neighbor_centroids):
    distances = cdist([current_centroid], neighbor_centroids)[0]
    min_distance = np.min(distances)
    min_distance_idx = np.argmin(distances)
    return min_distance, min_distance_idx

# Function to merge regions into a single label ID in the 3D label array
def merge_regions_3D(zstack_3D_z, label_id1, label_id2):
    zstack_3D_z[zstack_3D_z == label_id1] = label_id2
    return zstack_3D_z

In [23]:
def combine_in_3D(file_name,input_pre,output_pre):
    #file_name = "../" + m_metadata_df["File"][ith_file]
    img = ND2Reader(file_name)
    img.bundle_axes = 'zyx'
    img.iter_axes = 'c'
    img_0 = img[0]
    zstack = np.zeros(img_0.shape)
    for z_plane in range(len(img_0)):
        label_image_filename = input_pre + "_z" + '{:02}'.format(z_plane) +'.tif'
        try:
            zstack[z_plane] = io.imread(label_image_filename)
        except:
            print("file cannot read: " + label_image_filename)
    # Initialize the 3D label array with the same shape as the zstack
    zstack_3D = np.zeros_like(zstack,dtype='int')

    # Iterate over the zstack
    for z in range(1, zstack.shape[0]):
        current_image = zstack[z]
        current_image[current_image!=0] = current_image[current_image!=0] + np.max(zstack_3D)
        previous_image = zstack_3D[z-1]
        zstack_3D[z] = current_image.copy()
        # Label regions in the current image
    #     labeled_array, num_features = label(current_image)
        label_list_current = np.unique(current_image)
        num_features = len(label_list_current) -1
        # Calculate centroid coordinates for regions in the current image
        current_centroids = []
        for label_id in label_list_current[1:]:
            region = current_image == label_id
            centroid = calculate_centroid(region)
            current_centroids.append(centroid)

        # Label regions in previous image
    #     labeled_array_previous, num_features_previous = label(previous_image)
        label_list_previous = np.unique(previous_image)
        num_features_previous = len(np.unique(previous_image))-1
        # Calculate centroid coordinates for regions in the previous image
        previous_centroids = []
        for label_id in label_list_previous[1:]:
            region = previous_image == label_id
            centroid = calculate_centroid(region)
            previous_centroids.append(centroid)
        if len(previous_centroids) == 0:
            continue
        # Find closest centroid in the previous image for each region in the current image
        for current_label_id in range(num_features):
            current_centroid = current_centroids[current_label_id]
            min_distance, min_distance_idx = find_closest_centroid(current_centroid, previous_centroids)
            closest_label_id = min_distance_idx + 1  # Add 1 to match label IDs

#                 min_distance_list.append(min_distance)
            if min_distance < centroid_distance_threshold:
                # Merge regions into a single label ID in the 3D label array
                tmp_label_currentz = merge_regions_3D(zstack_3D[z].copy(), label_list_current[current_label_id+1], label_list_previous[closest_label_id])
                zstack_3D[z] = tmp_label_currentz
    ## filter the 3D objects based on intensity and volume
#         volume_list = []
#         intensity_list = [] # mean intensity of the 3D object
    object_3D_list = np.unique(zstack_3D)[1:].tolist()
    zstack_3D_filtered = np.zeros_like(zstack_3D)
    for ith_object in object_3D_list:
        volume_current = np.sum(zstack_3D==ith_object) * pixel_size * pixel_size * z_step_size #volume unit: um3
        intensity_current = np.mean(img_0[zstack_3D==ith_object])
#             volume_list.append(volume_current)
#             intensity_list.append(intensity_current)
        # filter the 3D label image and only keep big and bright nucleus region
        if volume_current>volume_threshold and intensity_current>intensity_threshold:
            zstack_3D_filtered[zstack_3D==ith_object] = ith_object

#         zstack_final, num_nucleus_final = label(zstack_3D_filtered)
    zstack_final = np.zeros_like(zstack_3D_filtered)
    ith_nucleus_3D_list = np.unique(zstack_3D_filtered)
    ith_nucleus_3D_list = ith_nucleus_3D_list[ith_nucleus_3D_list!=0]
    for ith_nucleus_3D in range(len(ith_nucleus_3D_list)):
        zstack_final[zstack_3D_filtered==ith_nucleus_3D_list[ith_nucleus_3D]] = ith_nucleus_3D + 1
    # io.imsave('test-3D-connect-centroids.tif',zstack_final.astype('uint16'))

    imwrite(output_pre + '-3D.tif', zstack_final.astype('uint16'), shape=zstack_final.shape,\
                    metadata={'axes': 'ZYX'}, imagej=True, photometric='minisblack', dtype='uint16', compression ='zlib')
#     io.imsave(output_pre + '-3D.tif',zstack_final.astype('uint16'))

In [24]:
num_threads = 8  # Number of threads to use

# Use joblib to run the process_file function in parallel with 8 threads
Parallel(n_jobs=num_threads)(
    delayed(combine_in_3D)(file_name=m_metadata_df["File"][ith_file], input_pre=os.path.join(path_segmentation, m_metadata_df["image_short"][ith_file]), output_pre=os.path.join(path_segmentation_3D_connect,m_metadata_df["image_short"][ith_file])) for ith_file in tqdm(range(len(m_metadata_df["File"].values)))
)

# # single thread
# for ith_file in tqdm(range(len(m_metadata_df["File"].values))):
#     combine_in_3D(file_name=os.path.join(".." , m_metadata_df["File"][ith_file]), input_pre=os.path.join(path_segmentation , m_metadata_df["image_short"][ith_file]), output_pre=os.path.join(path_segmentation_3D_connect, m_metadata_df["image_short"][ith_file]))

100%|███████████████████████████████████████████| 21/21 [00:59<00:00,  2.85s/it]


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]